In [2]:
import pandas as pd
import numpy as np
import config
import sql_con

In [3]:
# get config module variables
ROOT_DIR = config.ROOT_DIR
select_records = sql_con.select_records
insert_records = sql_con.insert_records
conn_odbc = sql_con.conn_odbc
read_contents = sql_con.read_contents

In [6]:
# read sales data
sales_df = pd.read_csv(rf"{ROOT_DIR}/data/sales.csv", index_col="Unnamed: 0")

In [7]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 1 to 178
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   product    178 non-null    object 
 1   client     178 non-null    object 
 2   status     178 non-null    object 
 3   amount     178 non-null    float64
 4   num_users  178 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 8.3+ KB


In [8]:
sales_df.head()

,product,client,status,amount,num_users
1,Product F,Current,Won,7389.52,19
2,Product C,New,Won,4493.01,43
3,Product B,New,Won,5738.09,87
4,Product I,Current,Won,2591.24,83
5,Product E,Current,Won,6622.97,17
